In [ ]:
# 📌 Instalar dependencias (si no están instaladas)
!pip install transformers torch scikit-learn numpy

In [ ]:
import json
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModel, AutoModelForSeq2SeqLM
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC

# Cargar modelos de Hugging Face
modelo_emb_name = "distilbert-base-uncased"
modelo_resp_name = "facebook/blenderbot-400M-distill"

tokenizer_emb = AutoTokenizer.from_pretrained(modelo_emb_name)
modelo_emb = AutoModel.from_pretrained(modelo_emb_name)

tokenizer_resp = AutoTokenizer.from_pretrained(modelo_resp_name)
modelo_resp = AutoModelForSeq2SeqLM.from_pretrained(modelo_resp_name)

# Cargar el dataset del chatbot (intents.json)
archivo_intents = "intents.json"
try:
    with open(archivo_intents, "r", encoding="utf-8") as file:
        intents = json.load(file)
except FileNotFoundError:
    intents = {"intents": []}

# Cargar dataset de celulares
archivo_celulares = "celulares_limpios.json"
try:
    with open(archivo_celulares, "r", encoding="utf-8") as file:
        celulares = json.load(file)
except FileNotFoundError:
    celulares = []

# Memoria de la conversación (para recordar contexto)
memoria_chat = []

def obtener_embedding(frase):
    tokens = tokenizer_emb(frase, return_tensors="pt", padding=True, truncation=True, max_length=64)
    with torch.no_grad():
        salida = modelo_emb(**tokens)
    return salida.last_hidden_state[:, 0, :].numpy().flatten()

patterns, labels = [], []

for intent in intents["intents"]:
    if "tag" in intent and "patterns" in intent:
        for pattern in intent["patterns"]:
            patterns.append(pattern)
            labels.append(intent["tag"])

if patterns:
    X = np.array([obtener_embedding(frase) for frase in patterns])
    label_encoder = LabelEncoder()
    y = label_encoder.fit_transform(labels)

    classifier = SVC(kernel="linear", probability=True)
    classifier.fit(X, y)
else:
    classifier = None

def predecir_intencion(frase):
    if classifier:
        embedding = obtener_embedding(frase)
        prediccion = classifier.predict([embedding])[0]
        return label_encoder.inverse_transform([prediccion])[0]
    return None

def obtener_respuesta_json(intencion):
    for intent in intents["intents"]:
        if intent["tag"] == intencion:
            return np.random.choice(intent["responses"])
    return None

def generar_respuesta_transformer(frase):
    tokens = tokenizer_resp(frase, return_tensors="pt", padding=True, truncation=True, max_length=64)
    with torch.no_grad():
        respuesta_ids = modelo_resp.generate(**tokens)
    return tokenizer_resp.decode(respuesta_ids[0], skip_special_tokens=True)

def respuesta_valida(respuesta):
    palabras_invalidas = ["no sé", "no entiendo", "no puedo ayudar", "no tengo información"]
    return not any(palabra in respuesta.lower() for palabra in palabras_invalidas)

def guardar_aprendizaje(nueva_pregunta, nueva_respuesta):
    global intents

    if not respuesta_valida(nueva_respuesta):
        return

    intencion_predicha = predecir_intencion(nueva_pregunta)

    if intencion_predicha:
        for intent in intents["intents"]:
            if intent["tag"] == intencion_predicha:
                if nueva_pregunta not in intent["patterns"]:
                    intent["patterns"].append(nueva_pregunta)
                if nueva_respuesta not in intent["responses"]:
                    intent["responses"].append(nueva_respuesta)
                break
    else:
        nueva_categoria = f"aprendizaje_{len(intents['intents'])+1}"
        intents["intents"].append({
            "tag": nueva_categoria,
            "patterns": [nueva_pregunta],
            "responses": [nueva_respuesta]
        })

    with open(archivo_intents, "w", encoding="utf-8") as f:
        json.dump(intents, f, indent=4, ensure_ascii=False)

def manejar_contexto(texto):
    global memoria_chat
    memoria_chat.append(texto)
    if len(memoria_chat) > 3:
        memoria_chat.pop(0)
    return " ".join(memoria_chat)

def responder_chatbot(texto):
    texto_con_contexto = manejar_contexto(texto)
    intencion = predecir_intencion(texto_con_contexto)
    respuesta = obtener_respuesta_json(intencion)

    if respuesta:
        return respuesta

    for celular in celulares:
        if any(palabra in texto.lower() for palabra in [celular["Company Name"].lower(), celular["Model Name"].lower()]):
            return json.dumps(celular, indent=4)

    respuesta_generada = generar_respuesta_transformer(texto_con_contexto)

    if respuesta_valida(respuesta_generada):
        guardar_aprendizaje(texto, respuesta_generada)

    return respuesta_generada

while True:
    pregunta = input(" Escribe tu consulta (o 'salir' para terminar): ")
    if pregunta.lower() == "salir":
        break
    print("", responder_chatbot(pregunta))